In [1]:
# Imports, paths e seed

import os, json, math, platform, warnings, time
from pathlib import Path
from datetime import datetime
from datetime import timezone
from itertools import product
from typing import List, Dict, Any

import numpy as np
import pandas as pd
from tqdm.auto import tqdm

from bertopic import BERTopic
from sentence_transformers import SentenceTransformer
from sklearn.feature_extraction.text import CountVectorizer
from umap import UMAP
from hdbscan import HDBSCAN
from gensim.models.coherencemodel import CoherenceModel

BASE = Path("../../data")
IN_CSV = BASE/"interim"/"bertopic"/"prep.csv"
OUT_DIR = BASE/"processed"/"bertopic"
FIGS = Path("../../reports/figs")

OUT_DIR.mkdir(parents=True, exist_ok=True)
FIGS.mkdir(parents=True, exist_ok=True)

SEED = 42
np.random.seed(SEED)

def _get_ver(pkg):
    try:
        import importlib.metadata as im
        return im.version(pkg)
    except Exception:
        return "NA"

versions = {
    "python": platform.python_version(),
    "bertopic": _get_ver("bertopic"),
    "sentence-transformers": _get_ver("sentence-transformers"),
    "umap-learn": _get_ver("umap-learn"),
    "hdbscan": _get_ver("hdbscan"),
    "scikit-learn": _get_ver("scikit-learn"),
    "gensim": _get_ver("gensim"),
    "numpy": _get_ver("numpy"),
    "pandas": _get_ver("pandas")
}
print("VERSIONS:", json.dumps(versions, indent=2, ensure_ascii=False))

VERSIONS: {
  "python": "3.12.2",
  "bertopic": "0.16.0",
  "sentence-transformers": "2.6.1",
  "umap-learn": "0.5.6",
  "hdbscan": "0.8.33",
  "scikit-learn": "1.4.2",
  "gensim": "4.3.3",
  "numpy": "1.26.4",
  "pandas": "2.2.2"
}


In [2]:
# Stopwords em português (para CountVectorizer)
try:
    from nltk.corpus import stopwords as nltk_stop
    STOP_WORDS_PT = sorted(set(nltk_stop.words('portuguese')))
    print(f"STOP_WORDS_PT carregadas: {len(STOP_WORDS_PT)} termos")
except Exception as e:
    # fallback mínimo para não travar caso NLTK falhe (mantém pipeline funcionando)
    print("[WARN] NLTK indisponível; usando fallback reduzido:", e)
    STOP_WORDS_PT = sorted(set("""
a à acerca agora ai ainda além algo alguem alguns algumas algum alguma ambos ambas ante antes ao aos após aquela aquelas aquele aqueles aquilo as assim até através cada quase com como contra contudo cujo cuja cujos cujas da das de dela delas dele deles depois desde desta deste disso disto do dos e é ela elas ele eles em entre era eram essa essas esse esses esta estas este estes estou eu foi foram fosse fossem fui há isso isto já la lá lhe lhes mais mas me mesmo mesmoa mesmos mesmas minha minhas meu meus muito muita muitas muitos não na nas nem no nos nós o os ou para pela pelas pelo pelos pouca poucas pouco poucos por porque porém pra qual quais quando que quem se sem sempre sendo ser seu seus sob sobre sua suas também tão tão tem tenho ter teu teus tua tuas tudo um uma umas uns
""".split()))

STOP_WORDS_PT carregadas: 207 termos


In [3]:
# Leitura e preparação dos textos

df = pd.read_csv(IN_CSV, encoding="utf-8")

required_cols = {"DOC_ID", "resumo", "RESUMO_PREP_BERTOPIC"}
missing = required_cols - set(df.columns)
assert not missing, f"Colunas ausentes: {missing}"

docs_for_embeddings = (
    df["resumo"].fillna(df["RESUMO_PREP_BERTOPIC"]).astype(str).tolist()
)
docs_for_vectorizer = (
    df["RESUMO_PREP_BERTOPIC"].fillna("").astype(str).tolist()
)

lens = pd.Series([len(x) for x in docs_for_vectorizer])
print("N documentos:", len(df))
print("Docs vazios (vectorizer):", sum([len(x.strip()) == 0 for x in docs_for_vectorizer]))
print("Comprimento (chars) — quantis:", lens.quantile([0.0, 0.25, 0.5, 0.75, 0.95, 1.0]).to_dict())


N documentos: 423
Docs vazios (vectorizer): 14
Comprimento (chars) — quantis: {0.0: 0.0, 0.25: 894.5, 0.5: 1123.0, 0.75: 1322.5, 0.95: 1729.2999999999997, 1.0: 2309.0}


In [4]:
# Embeddings (SBERT)

# Modelo multilíngue leve que preserva semântica de sentenças (SBERT)
# BERT → base dos embeddings contextuais [Devlin et al.] usados em SBERT
import torch
device = "cuda" if torch.cuda.is_available() else "cpu"
sbert_name = "paraphrase-multilingual-MiniLM-L12-v2"

start = time.time()
sbert = SentenceTransformer(sbert_name, device=device)
embeddings = sbert.encode(
    docs_for_embeddings,
    batch_size=64,
    show_progress_bar=True,
    normalize_embeddings=True,
    convert_to_numpy=True
)
elapsed = time.time() - start
print(f"Embeddings shape={embeddings.shape} | device={device} | time={elapsed:.1f}s")


Batches:   0%|          | 0/7 [00:00<?, ?it/s]

Embeddings shape=(423, 384) | device=cpu | time=11.2s


In [5]:
# Grade de hiperparâmetros (mini-sweep)

from math import floor
N_DOCS = len(docs_for_vectorizer)

# --- grade atualizada ---
grid_umap = {
    "n_neighbors": [10, 15, 30],
    "n_components": [5],
    "min_dist": [0.0],
    "metric": ["cosine"],
    "random_state": [SEED],
}
grid_hdb = {
    "min_cluster_size": [10, 15, 20],
    "min_samples": [None],
    "metric": ["euclidean"],
    "cluster_selection_method": ["eom"],
    "prediction_data": [True],
}
# >>> use a lista STOP_WORDS_PT (construída no Bloco 1B via NLTK ou spaCy)
grid_vec = {
    "ngram_range": [(1,1), (1,2)],
    "stop_words": [STOP_WORDS_PT],   # <<< nunca string "portuguese"
    "min_df": [2, 3],
    "max_df": [0.9],                 # vamos filtrar combos inválidos logo abaixo
}

# Produto cartesiano
from itertools import product
combos_all = []
for u, h, v in product(
    product(*grid_umap.values()),
    product(*grid_hdb.values()),
    product(*grid_vec.values())
):
    umap_kwargs = dict(zip(grid_umap.keys(), u))
    hdb_kwargs = dict(zip(grid_hdb.keys(), h))
    vec_kwargs = dict(zip(grid_vec.keys(), v))
    combos_all.append({"umap": umap_kwargs, "hdb": hdb_kwargs, "vec": vec_kwargs})

# Baseline com stopwords corretas
baseline = {
    "umap": {"n_neighbors": 15, "n_components": 5, "min_dist": 0.0, "metric": "cosine", "random_state": SEED},
    "hdb": {"min_cluster_size": 15, "min_samples": None, "metric": "euclidean",
            "cluster_selection_method": "eom", "prediction_data": True},
    "vec": {"ngram_range": (1,2), "stop_words": STOP_WORDS_PT, "min_df": 3, "max_df": 0.9},
}

# Move baseline para o início (sem duplicar)
def _same(a, b): return json.dumps(a, sort_keys=True) == json.dumps(b, sort_keys=True)
combos = [baseline] + [c for c in combos_all if not _same(c, baseline)]

# ---- filtro para evitar o erro "max_df corresponds to < documents than min_df"
def _valid_vec(v, n_docs: int) -> bool:
    min_df = v["min_df"]
    max_df = v["max_df"]
    # max_df como proporção → #docs máximos admitidos
    max_docs = floor(max_df * n_docs) if isinstance(max_df, float) else int(max_df)
    return max_docs >= int(min_df)

combos = [c for c in combos if _valid_vec(c["vec"], N_DOCS)]

# Opcional: limitar a 18 amostras determinísticas, mantendo baseline na frente
rng = np.random.RandomState(SEED)
if len(combos) > 18:
    # preserva baseline + 17 amostras
    others = combos[1:]
    idx = rng.choice(len(others), size=17, replace=False)
    combos = [combos[0]] + [others[i] for i in sorted(idx)]

print(f"Combos válidos após filtro: {len(combos)}")
print("Baseline stop_words tipo:", type(combos[0]['vec']['stop_words']).__name__,
      "| tamanho:", len(combos[0]['vec']['stop_words']) if hasattr(combos[0]['vec']['stop_words'], '__len__') else 'NA')

Combos válidos após filtro: 18
Baseline stop_words tipo: list | tamanho: 207


In [6]:
# Funções auxiliares

def write_json(path: Path, data: Dict[str, Any]):
    path.parent.mkdir(parents=True, exist_ok=True)
    with open(path, "w", encoding="utf-8") as f:
        json.dump(data, f, indent=2, ensure_ascii=False)

def extract_topic_words(topic_model: BERTopic, topk: int = 10) -> List[List[str]]:
    topics_dict = topic_model.get_topics()
    topic_words: List[List[str]] = []
    for tid, pairs in topics_dict.items():
        if tid == -1:
            continue
        # BERTopic pode retornar None/[] para tópicos degenerados
        if not pairs:
            continue
        # Garante apenas strings não vazias; respeita topk
        toks = [str(w).strip() for (w, _) in pairs[:topk] if isinstance(w, (str, bytes)) and str(w).strip()]
        if toks:
            topic_words.append(toks)
    return topic_words

def simple_tokenize_docs(docs: List[str]) -> List[List[str]]:
    # Tokenização simples coerente com o CountVectorizer (lower/space)
    return [str(d).lower().split() for d in docs]

def topic_diversity(topic_words: List[List[str]], topk: int = 10) -> float:
    if not topic_words:
        return float("nan")
    uniq = len(set([w for tw in topic_words for w in tw[:topk]]))
    return uniq / (topk * len(topic_words))

def outlier_rate(topics: List[int]) -> float:
    topics = np.array(topics)
    return float(np.mean(topics == -1))

def _sanitize_topics_for_gensim(topic_words: List[List[str]]) -> List[List[str]]:
    clean: List[List[str]] = []
    for tw in topic_words:
        if not isinstance(tw, (list, tuple)):
            continue
        toks = [str(t).strip() for t in tw if isinstance(t, (str, bytes)) and str(t).strip()]
        if toks:
            clean.append(toks)
    return clean

def compute_coherences(topic_words, tokenized_docs):
    # c_npmi e c_uci com 'texts' (ambas aceitam texts); sanear antes de passar ao gensim
    from gensim.corpora import Dictionary
    from gensim.models.coherencemodel import CoherenceModel

    cleaned = _sanitize_topics_for_gensim(topic_words)
    if not cleaned:
        return float("nan"), float("nan")

    dictionary = Dictionary(tokenized_docs)

    c_npmi = CoherenceModel(
        topics=cleaned, texts=tokenized_docs, dictionary=dictionary, coherence="c_npmi"
    ).get_coherence()

    c_uci = CoherenceModel(
        topics=cleaned, texts=tokenized_docs, dictionary=dictionary, coherence="c_uci"
    ).get_coherence()

    return float(c_npmi), float(c_uci)

In [7]:
 # Loop do sweep: treino, avaliação e salvamento por trial

tokenized_docs = simple_tokenize_docs(docs_for_vectorizer)
trials_summary = []
errors_log = []

N_DOCS_EFF = len(docs_for_vectorizer)  # usado para sanear min_df/max_df

for i, cfg in enumerate(tqdm(combos, desc="Sweep")):
    trial_id = f"trial_{i:02d}"
    tdir = OUT_DIR / trial_id
    tdir.mkdir(parents=True, exist_ok=True)

    # --- SANEAMENTO do Vectorizer (evita "max_df corresponds to < documents than min_df" e 'portuguese')
    vec_kwargs = cfg["vec"].copy()
    sw = vec_kwargs.get("stop_words", None)
    if isinstance(sw, str):
        vec_kwargs["stop_words"] = STOP_WORDS_PT

    min_df = vec_kwargs.get("min_df", 1)
    max_df = vec_kwargs.get("max_df", 1.0)
    min_df_prop = (min_df / max(N_DOCS_EFF, 1)) if isinstance(min_df, int) else float(min_df)
    max_df_prop = (max_df / max(N_DOCS_EFF, 1)) if isinstance(max_df, int) else float(max_df)
    if max_df_prop < min_df_prop:
        max_df_prop = min(1.0, max(min_df_prop + 1e-9, 0.999))
    vec_kwargs["min_df"] = min_df_prop
    vec_kwargs["max_df"] = max_df_prop

    # Instâncias dos componentes
    vectorizer_model = CountVectorizer(**vec_kwargs)
    umap_model = UMAP(**cfg["umap"])
    hdbscan_model = HDBSCAN(**cfg["hdb"])

    topic_model = BERTopic(
        embedding_model=None,
        vectorizer_model=vectorizer_model,
        umap_model=umap_model,
        hdbscan_model=hdbscan_model,
        calculate_probabilities=True,
        verbose=False
    )

    started = datetime.now(timezone.utc).isoformat()
    t0 = time.time()
    try:
        topics, probs = topic_model.fit_transform(
            documents=docs_for_vectorizer,
            embeddings=embeddings
        )
        fit_time = time.time() - t0

        # Artefatos primários
        topic_info = topic_model.get_topic_info()
        topic_info.to_csv(tdir/"topic_info.csv", index=False, encoding="utf-8")

        # Doc→tópico
        if probs is not None and hasattr(probs, "__array__"):
            rowmax = np.nanmax(probs, axis=1)
            doc_prob = [float(x) if np.isfinite(x) else np.nan for x in rowmax]
        else:
            doc_prob = [np.nan] * len(topics)

        doc_topics = pd.DataFrame({
            "DOC_ID": df["DOC_ID"].values,
            "topic": topics,
            "prob": doc_prob
        })
        doc_topics.to_csv(tdir/"doc_topics.csv", index=False, encoding="utf-8")

        ctf = topic_model.c_tf_idf_
        arr = ctf.toarray() if hasattr(ctf, "toarray") else np.asarray(ctf)
        np.save(tdir/"c_tf_idf.npy", arr)

        vocab = vectorizer_model.get_feature_names_out()
        with open(tdir/"vocab.txt", "w", encoding="utf-8") as f:
            f.write("\n".join(map(str, vocab)))


        # Métricas (com blindagem de coerência)
        topic_words = extract_topic_words(topic_model, topk=10)
        div = topic_diversity(topic_words, topk=10)
        try:
            c_npmi, c_uci = compute_coherences(topic_words, tokenized_docs)
        except Exception as ce:
            c_npmi, c_uci = float("nan"), float("nan")
            write_json(tdir/"coherence_error.json", {"error": repr(ce)})

        out_rate = outlier_rate(topics)

        metrics = {
            "topic_diversity@10": div,
            "c_npmi": c_npmi,
            "c_uci": c_uci,
            "outlier_rate": out_rate,
            "n_topics_excl_-1": int((topic_info["Topic"] != -1).sum()),
            "fit_time_sec": fit_time
        }
        write_json(tdir/"metrics.json", metrics)

        # Metadados de execução
        run_md = {
            "trial_id": trial_id,
            "started_utc": started,
            "finished_utc": datetime.now(timezone.utc).isoformat(),
            "seed": SEED,
            "versions": versions,
            "paths": {
                "input_csv": str(IN_CSV.resolve()),
                "trial_dir": str(tdir.resolve())
            },
            "params": {
                "umap": cfg["umap"],
                "hdbscan": cfg["hdb"],
                "vectorizer": {
                    **{k: (list(v) if k=="ngram_range" else v) for k, v in cfg["vec"].items()},
                    "_effective_min_df": vec_kwargs["min_df"],
                    "_effective_max_df": vec_kwargs["max_df"],
                },
                "sbert_model": sbert_name,
            },
            "sizes": {
                "n_docs": len(df),
                "emb_dim": int(embeddings.shape[1])
            }
        }
        write_json(tdir/"run_metadata.json", run_md)

        trials_summary.append({
            "trial_id": trial_id,
            "metrics": metrics,
            "cfg": cfg
        })

    except Exception as e:
        err = {"trial_id": trial_id, "error": repr(e)}
        errors_log.append(err)
        write_json(tdir/"error.json", err)
        print(f"[WARN] {trial_id} falhou: {e}")

Sweep:   0%|          | 0/18 [00:00<?, ?it/s]

In [8]:
# Ranking e escolha da melhor configuração

# Ranking composto (ordenação por: c_npmi desc, topic_diversity desc, outlier_rate asc)
if not trials_summary:
    raise RuntimeError("Nenhum trial concluído. Verifique errors_log e configurações.")

df_rank = pd.DataFrame([{
    "trial_id": t["trial_id"],
    "c_npmi": t["metrics"]["c_npmi"],
    "topic_diversity": t["metrics"]["topic_diversity@10"],
    "outlier_rate": t["metrics"]["outlier_rate"],
    "n_topics": t["metrics"]["n_topics_excl_-1"]
} for t in trials_summary])

df_rank["r1"] = df_rank["c_npmi"].rank(ascending=False, method="min")
df_rank["r2"] = df_rank["topic_diversity"].rank(ascending=False, method="min")
df_rank["r3"] = df_rank["outlier_rate"].rank(ascending=True, method="min")
df_rank["rank_sum"] = df_rank[["r1","r2","r3"]].sum(axis=1)

df_rank = df_rank.sort_values(["rank_sum", "r1", "r2", "r3"]).reset_index(drop=True)
best_trial_id = df_rank.iloc[0]["trial_id"]
best_row = df_rank.iloc[0].to_dict()

print("TOP-5 trials por critério composto:")
display(df_rank.head(5))

best_cfg = next(t["cfg"] for t in trials_summary if t["trial_id"] == best_trial_id)
print("Vencedora:", best_trial_id, "| cfg:", best_cfg)


TOP-5 trials por critério composto:


,trial_id,c_npmi,topic_diversity,outlier_rate,n_topics,r1,r2,r3,rank_sum
0,trial_01,-0.054650,1.0,0.0,2,1.0,1.0,1.0,3.0
1,trial_08,-0.131737,1.0,0.0,2,2.0,1.0,1.0,4.0
2,trial_14,-0.131737,1.0,0.0,2,2.0,1.0,1.0,4.0
3,trial_06,-0.131737,0.7,0.0,2,2.0,16.0,1.0,19.0
4,trial_07,-0.131737,0.7,0.0,2,2.0,16.0,1.0,19.0


Vencedora: trial_01 | cfg: {'umap': {'n_neighbors': 10, 'n_components': 5, 'min_dist': 0.0, 'metric': 'cosine', 'random_state': 42}, 'hdb': {'min_cluster_size': 10, 'min_samples': None, 'metric': 'euclidean', 'cluster_selection_method': 'eom', 'prediction_data': True}, 'vec': {'ngram_range': (1, 1), 'stop_words': ['a', 'ao', 'aos', 'aquela', 'aquelas', 'aquele', 'aqueles', 'aquilo', 'as', 'até', 'com', 'como', 'da', 'das', 'de', 'dela', 'delas', 'dele', 'deles', 'depois', 'do', 'dos', 'e', 'ela', 'elas', 'ele', 'eles', 'em', 'entre', 'era', 'eram', 'essa', 'essas', 'esse', 'esses', 'esta', 'estamos', 'estar', 'estas', 'estava', 'estavam', 'este', 'esteja', 'estejam', 'estejamos', 'estes', 'esteve', 'estive', 'estivemos', 'estiver', 'estivera', 'estiveram', 'estiverem', 'estivermos', 'estivesse', 'estivessem', 'estivéramos', 'estivéssemos', 'estou', 'está', 'estávamos', 'estão', 'eu', 'foi', 'fomos', 'for', 'fora', 'foram', 'forem', 'formos', 'fosse', 'fossem', 'fui', 'fôramos', 'fôssem